In [17]:
import json
with open("raw_data/telegram_messages.json") as f:
    json_data = json.load(f)
    json_data = json_data[::-1]
    print(type(json_data))

prepared_data = {}
prepared_data["anon_text"] = {}
prepared_data["user"] = {}
prepared_data["ts"] = {}
prepared_data["type"] = {}


<class 'list'>


In [18]:
from datetime import datetime, timezone
import numpy as np

timestamps = []
for index, message in enumerate(json_data):
    message_id = str(index)
    if message["text"] is None:
        continue
    prepared_data["anon_text"][message_id] = message["text"]
    prepared_data["user"][message_id] = message['sender'][str('id')]
    prepared_data["type"][message_id] = "message" 
    timestamps.append(datetime.fromisoformat(message['date']).timestamp())

timestamps = np.array(timestamps)
mean = np.mean(timestamps)
var = np.var(timestamps)
timestamps -= mean
timestamps /= var


# print(timestamps[0])
global_index = 0
for index, message in enumerate(json_data):
    message_id = str(message["id"])
    if message["text"] is None:
        continue
    prepared_data["ts"][index] = float(timestamps[global_index])
    global_index += 1 

with open("raw_data/telegram_prepared.json", "w") as f:
    json.dump(prepared_data, f)

In [19]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('universal_tagset')


[nltk_data] Downloading package punkt_tab to /home/s1elu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/s1elu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/s1elu/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [20]:
# write data to prepared_topics.json

from topic_segmenter.Runner import Runner

runner = Runner("raw_data/telegram_prepared.json")

allTopics = {}

userIds = dict()
userIndex = 0
for index, topic in enumerate(runner.run()):
    currentTopic = []
    for message in topic.getMessages():
        userId = message.getAuthor()
        if userId not in userIds:
            userIds[userId] = userIndex
            userIndex += 1

        userId = userIds[userId]
        currentTopic.append((userId, message.getText()))
    
    allTopics[index] = currentTopic
    index += 1


Processing message id: 0
Processing message id: 1
Processing message id: 2
Processing message id: 3
Processing message id: 4
Processing message id: 5
Processing message id: 6
Processing message id: 7
Processing message id: 8
Processing message id: 9
Processing message id: 10
Processing message id: 11
Processing message id: 12
Processing message id: 13
Processing message id: 14
Processing message id: 15
Processing message id: 16
Processing message id: 17
Processing message id: 18
Processing message id: 19
Processing message id: 20
Processing message id: 21
Processing message id: 22
Processing message id: 23
Processing message id: 24
Processing message id: 25
Processing message id: 26
Processing message id: 27
Processing message id: 28
Processing message id: 29
Processing message id: 30
Processing message id: 31
Processing message id: 32
Processing message id: 33
Processing message id: 34
Processing message id: 35
Processing message id: 36
Processing message id: 37
Processing message id:

In [21]:
import json

with open("raw_data/preprocessed_topics.json", "w", encoding="utf-8") as f:
    json.dump(allTopics, f, ensure_ascii=False)